In [1]:
#Import
from tqdm import tqdm
import pandas as pd
import regex as re
import os
from A_clean_quran_punc_eng import rmv_Quran_punct_num_eng
from B_read_trans_franco_match import franco_match, filter_match

### <font color="yellow"> 1. filter Quran , punct , Eng

In [2]:
Main_path = "/home/mmaher/CS_Clean/" 
name = "ASR/"
words_XQuran_Xeng_Xpunct = rmv_Quran_punct_num_eng(Main_path + name +  '/data_all_final.txt')

len all data :  1246968


100%|██████████| 1246968/1246968 [00:01<00:00, 775552.34it/s]


len all words :  334354
len quran words :  14872


100%|██████████| 334354/334354 [01:06<00:00, 5026.37it/s]


len words Xquran :  326471
len words Xquran_Xpunct :  326276
len words Xquran_Xpunct_Xeng :  325305


In [8]:
#Save filter
num = 1
for i in range(0,len(words_XQuran_Xeng_Xpunct),131715):
    df = pd.DataFrame()
    df[0] = words_XQuran_Xeng_Xpunct[i:i+131715]

    try:
        writer = pd.ExcelWriter( Main_path +  name + 'ar/ar'  + str(num) + '.xlsx', engine='xlsxwriter')
    except:
        os.mkdir(Main_path + name + '/ar/')
        writer = pd.ExcelWriter( Main_path +  name + 'ar/ar'  +  str(num) + '.xlsx', engine='xlsxwriter')
    
    df.to_excel(writer, sheet_name='welcome', index=False)
    writer.save()
    num+=1

try: os.mkdir(Main_path + name + '/en/')
except: pass

### <font color="yellow"> 2. Read Trans

In [10]:
words_XQuran_Xeng_Xpunct_trans = []
for i in range(1,1+len(os.listdir(Main_path  + name + "/en/"))):
    df = pd.read_excel(Main_path + name +  "/en/ar" + str(i) + ".xlsx")
    words_XQuran_Xeng_Xpunct_trans.extend(df[0]) # 0 is column name

### <font color="yellow"> 3. Franco & Match

In [17]:
words_Xquran_Xpunct_Xeng_trans_franco , matches = franco_match(words_XQuran_Xeng_Xpunct, words_XQuran_Xeng_Xpunct_trans)

  8%|▊         | 26633/325305 [00:00<00:02, 134064.58it/s]

nan
nan


 51%|█████     | 166350/325305 [00:01<00:01, 140538.83it/s]

nan


100%|██████████| 325305/325305 [00:05<00:00, 58593.55it/s]

len(words_Xquran_Xpunct_Xeng) 325305
len(words_Xquran_Xpunct_Xeng_trans) 325305
len(words_Xquran_Xpunct_Xeng_trans_franco) 325305
len(matches) 325305


### <font color="yellow"> 3. Filter Match > 0.8 & Not Eng

In [18]:
filtered_ar, filtered_en ,  filtered_fr, filtered_ma = filter_match(0.8,words_XQuran_Xeng_Xpunct,words_XQuran_Xeng_Xpunct_trans,words_Xquran_Xpunct_Xeng_trans_franco,matches)

100%|██████████| 325305/325305 [01:08<00:00, 4765.40it/s]

len after filter match > 0.8 : 3649


### <font color="yellow"> 4. Save

In [19]:
lists = [ words_XQuran_Xeng_Xpunct, words_XQuran_Xeng_Xpunct_trans, words_Xquran_Xpunct_Xeng_trans_franco, matches, filtered_ar, filtered_en, filtered_fr , filtered_ma]
names = [ "words_XQuran_Xeng_Xpunct", "words_XQuran_Xeng_Xpunct_trans", "words_Xquran_Xpunct_Xeng_trans_franco", "matches", "filtered_ar", "filtered_en", "filtered_fr" , "filtered_ma"]

In [20]:
try:os.mkdir( Main_path + name + "lists/" ) 
except:pass

for i in range(len(names)):
    textfile = open(Main_path + name + "lists/" + names[i] + ".txt", "w")
    for element in lists[i]:
        textfile.write(str(element) + "\n")
    textfile.close()

In [22]:
textfile = open(Main_path + name + "lists/filtered_ar_en_" + name[:-1] + ".txt", "w")
for e in range(0,len(filtered_ar)):
    textfile.write(filtered_ar[e] + "  @@  " + filtered_en[e] + "\n")
textfile.close()


df = pd.DataFrame()
df["Arabic words"] = filtered_ar
df["English words"] = filtered_en
df["Franco words"] = filtered_fr
df["matches"] = filtered_ma
df["check"] = [0]*len(filtered_ar)

writer = pd.ExcelWriter(  Main_path  + name + "/"  + 'ar_en_fr_ma_' + name[:-1] + '.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()